In [1]:
import sys
import functools
import time
from datetime import datetime
import cv2
import imutils
import numpy as np
import tensorflow as tf
from PyQt5.QtCore import Qt, QUrl, QTimer, QCoreApplication
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent
from PyQt5.QtMultimediaWidgets import QVideoWidget
from PyQt5.QtWidgets import QApplication, QLabel, QPushButton, QVBoxLayout, QHBoxLayout, QWidget, QSizePolicy

In [2]:
def create_traffic_light():
    traffic_light = QLabel()
    traffic_light.setFixedSize(100, 200)
    traffic_light.setAlignment(Qt.AlignCenter)
    return traffic_light

In [3]:
def update_traffic_light(traffic_light, state):
    if state == 'red':
        traffic_light.setStyleSheet("background-color: red;")
    elif state == 'green':
        traffic_light.setStyleSheet("background-color: green;")

In [4]:
def load_model():
    model = tf.keras.models.load_model('rcnn_epoch-02_val_loss-0.10.h5')
    return model

In [5]:
def preprocess_image(image):
    resized_image = cv2.resize(image, (224, 224))
    rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    normalized_image = rgb_image / 255.0
    processed_image = np.expand_dims(normalized_image, axis=0)
    return processed_image

In [6]:
def fairness():
    if (datetime.now().day == last_green.day and
        datetime.now().month == last_green.month and
        datetime.now().year == last_green.year and
        (datetime.now() - last_green).total_seconds() < max_minutes * 60):
        return False
    return True

In [7]:
def timing():
    if fairness():
        last_green = datetime.now()
        return True
    print("Not good time")
    return False

In [8]:
def find_marker(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    return cv2.minAreaRect(c)

In [9]:
def distance_to_camera(perWidth):
    return (KNOWN_DISTANCE * focalLength) / perWidth

In [10]:
def color_similarity(image, box):
    x, y, w, h = box
    roi = image[y:y+h, x:x+w]
    mean_color_box = np.mean(roi, axis=(0, 1))
    mean_color_object = [120, 100, 50]
    distance = np.linalg.norm(mean_color_box - mean_color_object)
    similarity = 1 / (1 + distance)
    return similarity

In [11]:
def waiting_for_me(frame, model):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    frame_rgb = cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_BGR2RGB)
    ss.setBaseImage(frame_rgb)    
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    best_box = None
    best_score = 0
    for result in ssresults:
        x, y, w, h = result
        timage = frame_rgb[y:y+h,x:x+w]
        resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
        img = np.expand_dims(resized, axis=0)
        out = model.predict(img)
        if out[0][1] < out[0][0]:
            score = color_similarity(frame_rgb, result[:4])
            combined_score = out[0][0] * score
            if combined_score > best_score:
                best_box = result
                best_score = score
    if best_box is not None:
        if len(best_box) > 1:
            centimeters = distance_to_camera(best_box[3])
            print("Distance detect: ", centimeters)
            if abs(centimeters - KNOWN_DISTANCE) == 30:
                if timing():
                    return best_box  
                else:
                    return None
            else:
                print("The pedestrian far or close")
    return None

In [12]:
def detect_pedestrian(frame, model):
    res = model.predict(frame)
    if res[0][1] < res[0][0]:
        return waiting_for_me(frame, model)
    else:
        return False

In [13]:
def capture_frame(media_player, video_path):
    current_time = media_player.position()
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_MSEC, current_time)
    ret, frame = cap.read()
    if ret:
        return frame
    return None

In [14]:
def play_video(media_player, video_url):
    media_player.setMedia(QMediaContent(video_url))
    media_player.setVolume(0)
    media_player.play()
    return

In [15]:
def process_frame(media_player, video_path, traffic_light, model):
    frame = capture_frame(media_player, video_path)
    if frame is not None:
        processed_image = preprocess_image(frame)
        pedestrian_detected = detect_pedestrian(processed_image, model)
        if pedestrian_detected is not None:
            update_traffic_light(traffic_light, 'green')
        else:
            print("No pedestrian detected")
            update_traffic_light(traffic_light, 'red')


In [16]:
def monitor_video(media_player, video_path, traffic_light, model, timer):
    if media_player.state() == QMediaPlayer.PlayingState:
        process_frame(media_player, video_path, traffic_light, model)
        QCoreApplication.processEvents()  

In [17]:
def exit_application(window):
    window.close()

In [22]:
def main():
    global last_green
    global max_minutes
    global KNOWN_DISTANCE
    global KNOWN_WIDTH
    global focalLength

    model = load_model()

    KNOWN_DISTANCE = 450.0
    KNOWN_WIDTH = 40.0
    image = cv2.imread("img.JPG")
    marker = find_marker(image)
    focalLength = (marker[1][0] * KNOWN_DISTANCE) / KNOWN_WIDTH

    last_green = datetime(1111, 1, 1, 1, 1, 1)
    max_minutes = 1

    app = QApplication(sys.argv)
    video_path = r"video_1.mp4"
    video_url = QUrl.fromLocalFile(video_path)

    window = QWidget()
    window.setWindowTitle("PyQt5 Video Player with Traffic Light")
    window.setGeometry(100, 100, 1200, 800)

    video_widget = QVideoWidget()
    video_widget.setMinimumSize(600, 400)
    video_widget.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

    media_player = QMediaPlayer(None, QMediaPlayer.VideoSurface)
    media_player.setVideoOutput(video_widget)

    traffic_light = QLabel()
    traffic_light.setFixedSize(100, 200)
    traffic_light.setAlignment(Qt.AlignCenter)
    update_traffic_light(traffic_light, 'red')


    exit_button = QPushButton("Exit")
    exit_button.clicked.connect(functools.partial(exit_application, window))

    layout = QHBoxLayout()
    layout.addWidget(video_widget)
    layout.addWidget(traffic_light)

    button_layout = QHBoxLayout()
    button_layout.addWidget(exit_button)

    main_layout = QVBoxLayout()
    main_layout.addLayout(layout)
    main_layout.addLayout(button_layout)

    window.setLayout(main_layout)

    timer = QTimer()
    timer.setInterval(5000)  
    timer.timeout.connect(lambda: process_frame(media_player, video_path, traffic_light, model))

    window.show()
    
    play_video(media_player, video_url)  
    timer.start() 

    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


1/1 [==============================] - 0s 260ms/step


SystemExit: 0